LOAD LIBRARIES

In [1]:
import pandas as pd
from datetime import datetime, timedelta

REQUESTS

In [2]:
import os
import glob
import pandas as pd
from datetime import datetime

# ───── CONSTANTS ─────
FILES_DIR      = "Files"
# Search for filenames containing the keywords
RC_KEYWORD = "RCsh"
LIQ_KEYWORD = "AcctLiq"

RC_FILE_NAME = next((f for f in os.listdir(FILES_DIR) if RC_KEYWORD in f and f.endswith('.xlsx')), None)
LIQ_FILE_NAME = next((f for f in os.listdir(FILES_DIR) if LIQ_KEYWORD in f and f.endswith('.xlsx')), None)

# Optional: raise error if not found
if not RC_FILE_NAME:
    raise FileNotFoundError(f"No Excel file containing '{RC_KEYWORD}' found in {FILES_DIR}")
if not LIQ_FILE_NAME:
    raise FileNotFoundError(f"No Excel file containing '{LIQ_KEYWORD}' found in {FILES_DIR}")

ACCOUNT_ID_CSV = os.path.join(FILES_DIR, "Constant", "AccountID.csv")

# Column names
RC_ACCT_COL      = 'Related Process: Account Number'
RC_GROSS_COL     = 'Related Process: Gross Amount Requested'
RC_MINSET_COL    = 'Related Process: Amount to be set aside'
PSC_MODEL_COL    = 'New Model Trading'

LIQ_ACCT_COL     = 'Related Process: Account Number'
TRADE_TYPE_COL   = 'Related Process: Trade Request Type(s)'
IS_CLOSING_COL   = 'Related Process: Is the Account closing?'

today_str = datetime.today().strftime('%m-%d-%Y')

# ───── Load Account-ID lookup ─────
acct_id_df = pd.read_csv(ACCOUNT_ID_CSV, dtype=str)
acct_id_df.rename(columns={'Account Number': 'ACCOUNT NUMBER'}, inplace=True)

# ───── 1) Process Raise-Cash file ─────
rc_path = os.path.join(FILES_DIR, RC_FILE_NAME)
if not os.path.exists(rc_path):
    raise FileNotFoundError(f"Cannot find {rc_path}")
df_rc = pd.read_excel(rc_path)

# A) Raise-Cash & Cash-Minimum
cash_rows = []
for _, r in df_rc.iterrows():
    acct = r.get(RC_ACCT_COL)
    if pd.isna(acct):
        continue

    gross = r.get(RC_GROSS_COL)
    if pd.notna(gross):
        cash_rows.append({
            'Account ID': None,
            'Account Number': str(acct),
            'Set Aside Type ($ or %)': '$',
            'Set Aside Amount': gross,
            'Set Aside Minimum': gross,
            'Set Aside Maximum': None,
            'Expiration Type': 'Transaction',
            'Expiration Date': None,
            'Transaction Type': 'Distribution/Merge Out',
            'Transaction Tolerance Band (%)': 40,
            'Percent Calculation Type': None,
            'Description': 'Raise Cash',
            'Start Date': today_str
        })

    minimum = r.get(RC_MINSET_COL)
    if pd.notna(minimum):
        cash_rows.append({
            'Account ID': None,
            'Account Number': str(acct),
            'Set Aside Type ($ or %)': '$',
            'Set Aside Amount': minimum,
            'Set Aside Minimum': minimum * 3,
            'Set Aside Maximum': None,
            'Expiration Type': None,
            'Expiration Date': None,
            'Transaction Type': None,
            'Transaction Tolerance Band (%)': None,
            'Percent Calculation Type': None,
            'Description': 'Cash Minimum',
            'Start Date': today_str
        })

# Export Raise Cash
out_cash_df = pd.DataFrame(cash_rows)
out_cash_df.to_excel("Raise_Cash_Requests.xlsx", index=False)
print(f"Generated {len(out_cash_df)} Raise Cash rows → Raise_Cash_Requests.xlsx")

# B) PSC / New-Model from same file
df_psc = df_rc[df_rc[PSC_MODEL_COL].notna()].copy()
df_psc[RC_ACCT_COL] = df_psc[RC_ACCT_COL].astype(str)
df_psc = df_psc.merge(
    acct_id_df[['ACCOUNT NUMBER','Account ID']],
    left_on=RC_ACCT_COL, right_on='ACCOUNT NUMBER', how='left'
)

psc_rows = []
for _, r in df_psc.iterrows():
    psc_rows.append({
        'Account Change History ID': 0,
        'Account ID': r['Account ID'],
        'Change Type': 'Aggregated Model',
        'Change Date': today_str,
        'Next': r[PSC_MODEL_COL],
        'Remove Account Change': None
    })

# ───── 2) Process Liquidation file ─────
liq_path = os.path.join(FILES_DIR, LIQ_FILE_NAME)
if not os.path.exists(liq_path):
    raise FileNotFoundError(f"Cannot find {liq_path}")
df_liq = pd.read_excel(liq_path)

df_liq = df_liq[df_liq[TRADE_TYPE_COL]=='Account Liquidation'].copy()
df_liq[LIQ_ACCT_COL] = df_liq[LIQ_ACCT_COL].astype(str)
df_liq = df_liq.merge(
    acct_id_df[['ACCOUNT NUMBER','Account ID']],
    left_on=LIQ_ACCT_COL, right_on='ACCOUNT NUMBER', how='left'
)

liq_rows = []
for _, r in df_liq.iterrows():
    next_flag = (
        'Liquidated Closed'
        if str(r[IS_CLOSING_COL]).strip().lower()=='yes'
        else 'Liquidated Open'
    )
    liq_rows.append({
        'Account Change History ID': 0,
        'Account ID': r['Account ID'],
        'Change Type': 'Aggregated Model',
        'Change Date': today_str,
        'Next': next_flag,
        'Remove Account Change': None
    })

# ───── 3) Combine PSC + Liquidation → Model_Changes.xlsx ─────
all_rows = psc_rows + liq_rows
ModelChanges = pd.DataFrame(all_rows, columns=[
    'Account Change History ID',
    'Account ID',
    'Change Type',
    'Change Date',
    'Next',
    'Remove Account Change'
])
ModelChanges.to_excel('Model_Changes.xlsx', index=False)
print(f"Exported {len(ModelChanges)} account changes → Model_Changes.xlsx")

# ───── 4) Build account_number_df with source tags ─────

# 4a) Raise Cash account numbers
rc_accts = [{'Account Numbers': str(acct), 'Source': 'Raise Cash'} for acct in out_cash_df['Account Number']]

# 4b) PSC / New-Model account numbers
psc_accts = [{'Account Numbers': str(acct), 'Source': 'PSC'} for acct in df_psc[RC_ACCT_COL]]

# 4c) Liquidation account numbers
liq_accts = [{'Account Numbers': str(acct), 'Source': 'Liquidation'} for acct in df_liq[LIQ_ACCT_COL]]

# Combine them
all_acct_records = rc_accts + psc_accts + liq_accts

# Create the DataFrame
account_number_df = pd.DataFrame(all_acct_records)

# Drop duplicates (keep first occurrence of source)
account_number_df = account_number_df.drop_duplicates(subset=['Account Numbers']).reset_index(drop=True)

# Display result
print(f"Collected {len(account_number_df)} account numbers → account_number_df")


c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Generated 18 Raise Cash rows → Raise_Cash_Requests.xlsx
Exported 2 account changes → Model_Changes.xlsx
Collected 19 account numbers → account_number_df


c:\Users\owen.osagiede\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ALERTS

In [3]:
from datetime import datetime
import os
import pandas as pd

# ───── after building account_number_df ─────
account_number_df.rename(columns={'Account Numbers': 'Account Number'}, inplace=True)

# Define folder path
folder_path = 'Files'
schwab_keyword = 'Alerts_Firm_TRILOGY CAPITAL'
lpl_keyword    = 'Notification'

def find_file_by_keyword(folder, keyword):
    for file in os.listdir(folder):
        if keyword in file and file.endswith('.csv'):
            return os.path.join(folder, file)
    raise FileNotFoundError(f"No file with keyword '{keyword}' found in folder '{folder}'.")

# Load files
schwab_noti_df = pd.read_csv(find_file_by_keyword(folder_path, schwab_keyword))
lpl_noti_df    = pd.read_csv(find_file_by_keyword(folder_path, lpl_keyword))

# 🏦 SCHWAB: Subject starts with "Move Money" and today's date
today = datetime.today().strftime('%m/%d/%Y')
schwab_noti_df = schwab_noti_df[
    schwab_noti_df['Subject'].str.startswith('Move Money', na=False) &
    schwab_noti_df['Date Created'].str.startswith(today)
]
schwab_noti_df['Account Number'] = (
    schwab_noti_df['Account']
    .str.replace('-', '', regex=False)
    .astype(str)
)
schwab_noti_df = schwab_noti_df[['Account Number']].copy()
schwab_noti_df['Source'] = 'SCHWAB Alert'

# 🏛️ LPL: Category == "Insufficient Funds"
lpl_noti_df = lpl_noti_df[lpl_noti_df['Category'] == 'Insufficient Funds']
lpl_noti_df['Account Number'] = (
    lpl_noti_df['Account']
    .str.replace('-', '', regex=False)
    .astype(str)
)
lpl_noti_df = lpl_noti_df[['Account Number']].copy()
lpl_noti_df['Source'] = 'LPL Alert'

# Combine alerts
noti_df = pd.concat([schwab_noti_df, lpl_noti_df], ignore_index=True)

# 1) Identify overlaps
existing = set(account_number_df['Account Number'])
incoming = set(noti_df['Account Number'])
overlap = existing & incoming

if overlap:
    print(f"⚠️ {len(overlap)} alert account(s) already present in master list: {sorted(overlap)}")
else:
    print("✅ No alert accounts were already in the master list.")

# 2) Append only new alert accounts with source
new_noti_df = noti_df[~noti_df['Account Number'].isin(existing)]
if not new_noti_df.empty:
    account_number_df = pd.concat([account_number_df, new_noti_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"📩 Appended {len(new_noti_df)} new alert account(s) to master list.")
else:
    print("No new alert accounts to append.")

# Optional: export final alert log
noti_df.to_csv('Notifications.csv', index=False)


✅ No alert accounts were already in the master list.
📩 Appended 27 new alert account(s) to master list.


REBALANCES

In [4]:
# 1) Load the OOM.csv file
oom_keyword = 'Accounts With Positions Not In'
oom_df = pd.read_csv(find_file_by_keyword(folder_path, oom_keyword))

# 2) Drop accounts already in master list
oom_df = oom_df[
    ~oom_df['Account #'].astype(int)
           .isin(account_number_df['Account Number'].astype(int))
]

# 3) Apply filters
oom_df = (
    oom_df[~oom_df['Account Model']
               .str.contains('Custom|Tailored|Legacy', na=False)]
          .dropna(subset=['Account Model'])
)
oom_df = oom_df[oom_df['Account Value'] >= 100]

bad_tickers = ('RSX', 'DEWM', 'CPUT', '9999227', 'SchwabCash', 'SWGXX')
oom_df = oom_df[~oom_df['Ticker'].isin(bad_tickers)]

oom_df = (
    oom_df[~oom_df['Account Model']
               .str.contains('Liquidated', na=False)]
          .dropna(subset=['Account Model'])
)
oom_df = oom_df[oom_df['Current Units'] >= 1]

# 4) Prepare rebalance DataFrame
oom_df.rename(columns={'Account #': 'Account Number'}, inplace=True)
oom_df['Account Number'] = oom_df['Account Number'].astype(int)

rebalance_df = oom_df[['Account Number']].copy()
rebalance_df['Source'] = 'Rebalances'
rebalance_df.to_csv('Rebalance.csv', index=False)
print(f"🌀 Generated {len(rebalance_df)} rebalance rows → Rebalance.csv")

# 5) Append to account_number_df only if new
existing = set(account_number_df['Account Number'].astype(int))
incoming = set(rebalance_df['Account Number'])
new_accounts = incoming - existing

if new_accounts:
    new_df = rebalance_df[rebalance_df['Account Number'].isin(new_accounts)]
    account_number_df = pd.concat([account_number_df, new_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"✅ Appended {len(new_accounts)} new rebalance account(s) to master list.")
else:
    print("No new rebalance accounts to append to master list.")

🌀 Generated 814 rebalance rows → Rebalance.csv
✅ Appended 166 new rebalance account(s) to master list.


CONTRIBUTIONS

In [5]:
# Keyword for Accounts file
accounts_keyword = 'AccountList'
accounts_df = pd.read_csv(find_file_by_keyword(folder_path, accounts_keyword), dtype=str)

# 1) Normalize Account Numbers
accounts_df['Account Number'] = (
    accounts_df['Account Number']
      .fillna('')
      .str.replace(r'\D', '', regex=True)
)
accounts_df = accounts_df[accounts_df['Account Number'] != ""]

# 2) Drop accounts already in master list
accounts_df = accounts_df[
    ~accounts_df['Account Number']
      .isin(account_number_df['Account Number'])
]

# 3) Filter unwanted models
accounts_df = (
    accounts_df[~accounts_df['Model']
                         .str.contains('Custom|Liquidated|Tailored', na=False)]
               .dropna(subset=['Model'])
)

# 4) Clean dollar columns
for col in ['Current Cash $', 'Set Aside Cash Target', 'Total Value']:
    accounts_df[col] = (
        accounts_df[col]
          .str.replace('[$,]', '', regex=True)
          .astype(float)
    )

# 5) Compute True Cash %
accounts_df['True Cash Value'] = (
    (accounts_df['Current Cash $'] - accounts_df['Set Aside Cash Target'])
    / accounts_df['Total Value']
)

# 6) Numeric filters
accounts_df = accounts_df[accounts_df['Total Value'] >= 100]
accounts_df = accounts_df[accounts_df['True Cash Value'] >= 0.03]

# 7) Prepare tagged contribution accounts
contrib_df = accounts_df[['Account Number']].copy()
contrib_df['Source'] = 'Contributions'
contrib_df.to_csv('Contributions.csv', index=False)
print(f"📥 Exported {len(contrib_df)} contribution accounts → Contributions.csv")

# 8) Append only new ones to master list
existing = set(account_number_df['Account Number'])
incoming = set(contrib_df['Account Number'])
new_df = contrib_df[contrib_df['Account Number'].isin(incoming - existing)]

if not new_df.empty:
    account_number_df = pd.concat([account_number_df, new_df], ignore_index=True)
    account_number_df = account_number_df.drop_duplicates(subset=['Account Number'])
    print(f"✅ Appended {len(new_df)} new contribution account(s) to master list.")
else:
    print("No new contribution accounts to append to master list.")

📥 Exported 2722 contribution accounts → Contributions.csv
✅ Appended 2722 new contribution account(s) to master list.


In [6]:
# 1) Reload AccountList file to ensure all rows are available
accounts_keyword = 'AccountList'
account_list_df = pd.read_csv(find_file_by_keyword(folder_path, accounts_keyword), dtype=str)

# 2) Clean and normalize account numbers
account_list_df['Account Number'] = (
    account_list_df['Account Number']
      .fillna('')
      .str.replace(r'\D', '', regex=True)
)

# 3) Convert Managed Value to numeric
account_list_df['Managed Value'] = (
    account_list_df['Managed Value']
      .str.replace('[$,]', '', regex=True)
      .astype(float)
)

# 4) Create a set of accounts with Managed Value >= $5
valid_accounts = set(
    account_list_df[account_list_df['Managed Value'] >= 5]['Account Number']
)

# 5) Filter account_number_df
pre_filter_count = len(account_number_df)
account_number_df = account_number_df[
    account_number_df['Account Number'].isin(valid_accounts)
].reset_index(drop=True)
post_filter_count = len(account_number_df)

# 6) Final export
account_number_df.to_csv('Master Account Numbers.csv', index=False)
print(f"✅ Exported {post_filter_count} accounts → Master Account Numbers.csv (filtered out {pre_filter_count - post_filter_count} under-$5 accounts)")


✅ Exported 2745 accounts → Master Account Numbers.csv (filtered out 189 under-$5 accounts)
